In [1]:
import numpy as np
import pandas as pd
import jsonlines
from tqdm import tqdm

***

In [11]:
train = pd.read_parquet("../data/train_data.parquet")
train["date"] = pd.to_datetime(train["date"])
train

,sku,date,sold_quantity,current_price,currency,listing_type,shipping_logistic_type,shipping_payment,minutes_active
0,464801,2021-02-01,0,156.78,REA,classic,fulfillment,free_shipping,1440.000000
1,464801,2021-02-02,0,156.78,REA,classic,fulfillment,free_shipping,1440.000000
2,464801,2021-02-03,0,156.78,REA,classic,fulfillment,free_shipping,1440.000000
3,464801,2021-02-04,0,156.78,REA,classic,fulfillment,free_shipping,1440.000000
4,464801,2021-02-05,1,156.78,REA,classic,fulfillment,free_shipping,1440.000000
...,...,...,...,...,...,...,...,...,...
37660274,129187,2021-03-31,0,22057.00,ARG,classic,drop_off,free_shipping,267.710767
37660275,6707,2021-03-31,0,26999.00,ARG,classic,cross_docking,free_shipping,266.083333
37660276,170355,2021-03-31,0,3400.00,ARG,classic,drop_off,paid_shipping,0.252633
37660277,246568,2021-03-31,0,6289.00,ARG,classic,fulfillment,free_shipping,135.416667


In [12]:
all_records = list()
with jsonlines.open('../data/items_static_metadata_full.jl') as reader:
    for obj in tqdm(reader):
        all_records.append(obj)
        
metadata = pd.DataFrame(all_records)
metadata.drop(["item_title"], axis=1, inplace=True)
metadata

660916it [00:02, 221808.98it/s]


,item_domain_id,item_id,site_id,sku,product_id,product_family_id
0,MLB-SNEAKERS,492155,MLB,0,None,MLB15832732
1,MLB-SURFBOARD_RACKS,300279,MLB,1,None,None
2,MLM-NECKLACES,69847,MLM,2,None,None
3,MLM-RINGS,298603,MLM,3,None,None
4,MLB-WEBCAMS,345949,MLB,4,None,None
...,...,...,...,...,...,...
660911,MLB-CELLPHONE_PARTS,320792,MLB,660911,None,None
660912,MLM-AUTOMOTIVE_EMBLEMS,90441,MLM,660912,None,None
660913,MLB-SOFA_AND_FUTON_COVERS,202580,MLB,660913,None,None
660914,MLB-SNEAKERS,490874,MLB,660914,None,None


In [13]:
dataset = pd.merge(train, metadata, how="inner", on="sku")
dataset

,sku,date,sold_quantity,current_price,currency,listing_type,shipping_logistic_type,shipping_payment,minutes_active,item_domain_id,item_id,site_id,product_id,product_family_id
0,464801,2021-02-01,0,156.78,REA,classic,fulfillment,free_shipping,1440.000000,MLB-NEBULIZERS,344151,MLB,MLB9838512,MLB9838510
1,464801,2021-02-02,0,156.78,REA,classic,fulfillment,free_shipping,1440.000000,MLB-NEBULIZERS,344151,MLB,MLB9838512,MLB9838510
2,464801,2021-02-03,0,156.78,REA,classic,fulfillment,free_shipping,1440.000000,MLB-NEBULIZERS,344151,MLB,MLB9838512,MLB9838510
3,464801,2021-02-04,0,156.78,REA,classic,fulfillment,free_shipping,1440.000000,MLB-NEBULIZERS,344151,MLB,MLB9838512,MLB9838510
4,464801,2021-02-05,1,156.78,REA,classic,fulfillment,free_shipping,1440.000000,MLB-NEBULIZERS,344151,MLB,MLB9838512,MLB9838510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37660274,129187,2021-03-31,0,22057.00,ARG,classic,drop_off,free_shipping,267.710767,MLA-RADIO_BASE_STATIONS,408559,MLA,None,None
37660275,6707,2021-03-31,0,26999.00,ARG,classic,cross_docking,free_shipping,266.083333,MLA-PRINTERS,193784,MLA,MLA15159034,MLA15159032
37660276,170355,2021-03-31,0,3400.00,ARG,classic,drop_off,paid_shipping,0.252633,MLA-WRISTWATCHES,110276,MLA,None,None
37660277,246568,2021-03-31,0,6289.00,ARG,classic,fulfillment,free_shipping,135.416667,MLA-HARD_DRIVES_AND_SSDS,456892,MLA,MLA15697725,MLA15697724


In [14]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37660279 entries, 0 to 37660278
Data columns (total 14 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   sku                     int64         
 1   date                    datetime64[ns]
 2   sold_quantity           int64         
 3   current_price           float64       
 4   currency                object        
 5   listing_type            object        
 6   shipping_logistic_type  object        
 7   shipping_payment        object        
 8   minutes_active          float64       
 9   item_domain_id          object        
 10  item_id                 int64         
 11  site_id                 object        
 12  product_id              object        
 13  product_family_id       object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(8)
memory usage: 4.2+ GB


In [15]:
dataset.to_parquet("../data/dataset.parquet", index=False)

***